# Downloading dataset
Attempting to download historical weather datasets by using [MSC GeoMet API](https://eccc-msc.github.io/open-data/msc-geomet/readme_en/)
Data should include temperature, rain, wind, coordinates

Querying WMS from MSC by using Regional Ensemble Prediction System layer `REPS.DIAG.3_PRMM.ERGE5` : contains the probability that the quantity of precipitations will be equal or greater than 5mm

In [2]:
# Importation of Python modules
from datetime import datetime, timedelta
import re
import warnings

# The following modules must first be installed to use
# this code out of Jupyter Notebook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy
from owslib.wms import WebMapService
import pandas
from tabulate import tabulate

# Ignore warnings from the OWSLib module
warnings.filterwarnings('ignore', module='owslib', category=UserWarning)

# Parameters choice
# Layer:
layer = 'REPS.DIAG.3_PRMM.ERGE5'
# Coordinates:
y, x = 49.288, -123.116
# Local time zone (in this exemple, the local time zone is UTC-07:00):
time_zone = -7

Bounding Box parameters `bbox`

In [3]:
# bbox parameter
min_x, min_y, max_x, max_y = x - 0.25, y - 0.25, x + 0.25, y + 0.25

In [4]:
# WMS service connection
wms = WebMapService('https://geo.weather.gc.ca/geomet?SERVICE=WMS' +
                    '&REQUEST=GetCapabilities',
                    version='1.3.0',
                    timeout=300)